In [ ]:
import pyspark
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

In [ ]:
df = spark.read.parquet('yellow/2024/10')

In [ ]:
# Count trips that only started on the 15th of october
from pyspark.sql import functions as F
from pyspark.sql.functions import to_date, lit

# Rename cols, convert to date time, filter to pickup_date of October 15
df = df\
        .withColumnRenamed('tpep_pickup_datetime', 'pickup_datetime') \
        .withColumnRenamed('tpep_dropoff_datetime', 'dropoff_datetime') \
        
df = df.withColumn('pickup_date', F.to_date(df.pickup_datetime))

df\
    .filter(df.pickup_date == lit('2024-10-15')) \
    .count()

In [ ]:
from pyspark.sql.functions import unix_timestamp, expr

df_with_duration = df.withColumn(
    "trip_duration_hours",
    (unix_timestamp("dropoff_datetime") - unix_timestamp("pickup_datetime")) / 3600
)

longest_trip = df_with_duration.agg({"trip_duration_hours": "max"}).collect()[0][0]

print(f"The longest trip duration is {longest_trip} hours")


In [ ]:
df_lookup = spark.read \
    .options(header=True, inferSchema=True) \
    .csv('taxi_zone_lookup.csv')
    

In [ ]:
df_lookup.registerTempTable('lookup_table')
df.registerTempTable('yellow_taxi')

In [ ]:
spark.sql(
    """
    SELECT
        Zone,
        COUNT(*)
    FROM yellow_taxi AS yt
    INNER JOIN lookup_table AS lt
        ON yt.PULocationID = lt.LocationID
    GROUP BY Zone
    ORDER BY 2 ASC;
    """
).show()